In [1]:
import csv
import pandas as pd
import datetime as dt

# 1. France

In [2]:
L_parc_hist=[]
Parc_hist_df=pd.read_csv("Nucleaire_historique.csv",sep=';',decimal=',')
for i in Parc_hist_df.index:
    row=list(Parc_hist_df.loc[i])
    L_parc_hist.append(row)

In [3]:
L_new_nuc=[]
New_nuc_df=pd.read_csv("Nouveau_nucleaire.csv",sep=';',decimal=',')
for i in New_nuc_df.index:
    row=list(New_nuc_df.loc[i])
    L_new_nuc.append(row)

In [6]:
# Key functions
L_modul_year=[1.2,1.1,1,0.95,0.95,0.9,0.95,0.93,0.9,1,1.02,1.1]
taux_unav_hist=0.27
taux_unav_new=0.17
Power_EPR2=1670
Power_SMR=340

def duration_unav(row):
    if row[0]=='Flamanville-3':
        return round(365*taux_unav_new)
    else:
        return round(365*taux_unav_hist)

def complete_parc(L_parc_hist,L_new_nuc,year,mix='nuclear_plus'):
    if mix=='100_enr' and year>=L_new_nuc[-1][0]:
        return []
    
    L_parc_year=[]
    if mix!='nuclear_plus_plus':
        for row in L_parc_hist:
            if year<=row[2]:
                L_parc_year.append(row+[duration_unav(row),None,None])
    else:
        for row in L_parc_hist:
            if year<=row[2]+10 and row[2]!=2020:
                L_parc_year.append(row+[duration_unav(row),None,None])
    
    if year<L_new_nuc[0][0]:
        N_EPR2=0
        N_SMR=0
    elif year>=L_new_nuc[-1][0]:
        if mix in ['nuclear_plus','nuclear_plus_plus']:
            N_EPR2=L_new_nuc[-1][1]
            N_SMR=L_new_nuc[-1][2]
        elif mix=='nuclear_minus':
            N_EPR2=L_new_nuc[-1][3]
            N_SMR=L_new_nuc[-1][4]
        else:
            N_EPR2=L_new_nuc[-1][5]
            N_SMR=L_new_nuc[-1][6]
    else:
        if mix in ['nuclear_plus','nuclear_plus_plus']:
            N_EPR2=L_new_nuc[year-L_new_nuc[0][0]][1]
            N_SMR=L_new_nuc[year-L_new_nuc[0][0]][2]
        elif mix=='nuclear_minus':
            N_EPR2=L_new_nuc[year-L_new_nuc[0][0]][3]
            N_SMR=L_new_nuc[year-L_new_nuc[0][0]][4]
        else:
            N_EPR2=L_new_nuc[year-L_new_nuc[0][0]][5]
            N_SMR=L_new_nuc[year-L_new_nuc[0][0]][6]
    
    for i in range(N_EPR2):
        L_parc_year.append(['EPR2',Power_EPR2,2100]+[round(365*taux_unav_new),None,None])
    for i in range(N_SMR):
        L_parc_year.append(['SMR',Power_SMR,2100]+[round(365*taux_unav_new),None,None])
    
    return L_parc_year

L_month=[31,28,31,30,31,30,31,31,30,31,30,31]
def goal_month(L_parc_year,L_modul_year):
    P_moy=sum([x[1]*(365-x[3])/365 for x in L_parc_year])
    return [L_month[m]*P_moy*24*L_modul_year[m] for m in range(12)]

def prod_month(L_parc_year,m):# m in [0..11] here
    E=0
    d0=dt.datetime(2019,m+1,1)
    if m<11:
        d1=dt.datetime(2019,m+2,1)
    else:
        d1=dt.datetime(2020,1,1)
    for x in L_parc_year:
        if x[4]==None or x[4]>=d1 or x[5]<=d0:
            E+=x[1]*L_month[m]*24
        else:
            delta=min(d1,x[5])-max(d0,x[4])
            E+=(L_month[m]*24-(delta.days*24+delta.seconds/3600))*x[1]
    return E

d2020=dt.datetime(2020,1,1)
d1h=dt.timedelta(hours=1)
def optimize(L_parc_hist,L_new_nuc,L_modul_year,year,mix='nuclear_plus',delta_plus=0.01,delta_minus=0.01):
    L_parc_year=complete_parc(L_parc_hist,L_new_nuc,year,mix)
    L_goals=goal_month(L_parc_year,L_modul_year)
    for m in range(12):
        d0=dt.datetime(2019,m+1,1)
        if m<11:
            d1=dt.datetime(2019,m+2,1)
        else:
            d1=d2020
        E_prod=prod_month(L_parc_year,m)
        i=0
        while i<len(L_parc_year) and E_prod>(1+delta_plus)*L_goals[m]:
            if L_parc_year[i][4]!=None:
                i+=1
            else:
                delta_arret_tranche=dt.timedelta(days=L_parc_year[i][3])
                L_parc_year[i][4]=d0
                L_parc_year[i][5]=d0+delta_arret_tranche
                if L_parc_year[i][5]>d2020:
                    delta_depasse=L_parc_year[i][5]-d2020
                    L_parc_year[i][4]-=delta_depasse
                    L_parc_year[i][5]=d2020
                E_prod=prod_month(L_parc_year,m)
                while E_prod<(1-delta_minus)*L_goals[m] and L_parc_year[i][5]<d2020:
                    L_parc_year[i][4]+=d1h
                    L_parc_year[i][5]+=d1h
                    E_prod=prod_month(L_parc_year,m)
                i+=1
    return L_parc_year

def nuc_capacity(L_parc_year):
    P_old_nuc=0
    P_new_nuc=0
    for x in L_parc_year:
        if x[0] in ['EPR2','SMR','Flamanville-3']:
            P_new_nuc+=x[1]
        else:
            P_old_nuc+=x[1]
    return P_new_nuc, P_old_nuc

d2019=dt.datetime(2019,1,1)
def nuc_availability(L_parc_year):
    P_new_nuc, P_old_nuc=nuc_capacity(L_parc_year)
    
    L_dates=[]
    L_new_nuc=[0 for i in range(8760)]
    L_old_nuc=[0 for i in range(8760)]
    date=d2019
    for i in range(8760):
        L_dates.append(pd.Timestamp(date))
        date+=d1h
        for x in L_parc_year:
            if not x[4]<=L_dates[-1]<=x[5]:
                if x[0] in ['EPR2','SMR','Flamanville-3']:
                    L_new_nuc[i]+=x[1]
                else:
                    L_old_nuc[i]+=x[1]
                    
    L_new_nuc=[x/max(P_new_nuc,1) for x in L_new_nuc]
    L_old_nuc=[x/max(P_old_nuc,1) for x in L_old_nuc]
    
    return pd.DataFrame({"Date":L_dates,"New nuke":L_new_nuc,"Old nuke":L_old_nuc})
    

In [7]:
# Nuclear availability and capacity
L_capa_plus_plus=[]
L_capa_plus=[]
L_capa_minus=[]
L_capa_100_enr=[]
for year in [2030,2040,2050,2060]:
    for mix in ['nuclear_plus_plus','nuclear_plus','nuclear_minus','100_enr']:
        L_parc_year=optimize(L_parc_hist,L_new_nuc,L_modul_year,year,mix)
        print(L_parc_year)
        Availability_df=nuc_availability(L_parc_year)
        if mix=='nuclear_plus_plus':
            L_capa_plus_plus.append(list(nuc_capacity(L_parc_year)))
        elif mix=='nuclear_plus':
            L_capa_plus.append(list(nuc_capacity(L_parc_year)))
        elif mix=='nuclear_minus':
            L_capa_minus.append(list(nuc_capacity(L_parc_year)))
        else:
            L_capa_100_enr.append(list(nuc_capacity(L_parc_year)))
        Availability_df.to_csv("availability_nuclear_"+str(year)+"_"+mix+".csv",sep=";",decimal=".",index=False)
Capa_plus_plus_df=pd.DataFrame({"Year":[2030,2040,2050,2060],
                           "New nuke":[x[0] for x in L_capa_plus_plus],"Old nuke":[x[1] for x in L_capa_plus_plus]})
Capa_plus_df=pd.DataFrame({"Year":[2030,2040,2050,2060],
                           "New nuke":[x[0] for x in L_capa_plus],"Old nuke":[x[1] for x in L_capa_plus]})
Capa_minus_df=pd.DataFrame({"Year":[2030,2040,2050,2060],
                           "New nuke":[x[0] for x in L_capa_minus],"Old nuke":[x[1] for x in L_capa_minus]})
Capa_100_enr_df=pd.DataFrame({"Year":[2030,2040,2050,2060],
                           "New nuke":[x[0] for x in L_capa_100_enr],"Old nuke":[x[1] for x in L_capa_100_enr]})
Capa_plus_plus_df.to_csv("Capacity_nuclear_plus_plus.csv",sep=";",decimal=".",index=False)
Capa_plus_df.to_csv("Capacity_nuclear_plus.csv",sep=";",decimal=".",index=False)
Capa_minus_df.to_csv("Capacity_nuclear_minus.csv",sep=";",decimal=".",index=False)
Capa_100_enr_df.to_csv("Capacity_100_enr.csv",sep=";",decimal=".",index=False)

[['Bugey-2', 910, 2029, 99, datetime.datetime(2019, 1, 1, 0, 0), datetime.datetime(2019, 4, 10, 0, 0)], ['Bugey-3', 910, 2029, 99, datetime.datetime(2019, 1, 1, 0, 0), datetime.datetime(2019, 4, 10, 0, 0)], ['Bugey-4', 880, 2039, 99, datetime.datetime(2019, 1, 1, 0, 0), datetime.datetime(2019, 4, 10, 0, 0)], ['Bugey-5', 880, 2040, 99, datetime.datetime(2019, 1, 1, 0, 0), datetime.datetime(2019, 4, 10, 0, 0)], ['Tricastin-1', 915, 2030, 99, datetime.datetime(2019, 1, 1, 0, 0), datetime.datetime(2019, 4, 10, 0, 0)], ['Tricastin-2', 915, 2030, 99, datetime.datetime(2019, 1, 1, 0, 0), datetime.datetime(2019, 4, 10, 0, 0)], ['Tricastin-3', 915, 2035, 99, datetime.datetime(2019, 1, 1, 0, 0), datetime.datetime(2019, 4, 10, 0, 0)], ['Tricastin-4', 915, 2035, 99, datetime.datetime(2019, 1, 1, 0, 0), datetime.datetime(2019, 4, 10, 0, 0)], ['Dampierre-1', 890, 2039, 99, datetime.datetime(2019, 2, 1, 0, 0), datetime.datetime(2019, 5, 11, 0, 0)], ['Dampierre-2', 890, 2040, 99, datetime.datetime(201

[['Bugey-4', 880, 2039, 99, datetime.datetime(2019, 1, 1, 0, 0), datetime.datetime(2019, 4, 10, 0, 0)], ['Bugey-5', 880, 2040, 99, datetime.datetime(2019, 1, 1, 0, 0), datetime.datetime(2019, 4, 10, 0, 0)], ['Tricastin-1', 915, 2030, 99, datetime.datetime(2019, 1, 1, 0, 0), datetime.datetime(2019, 4, 10, 0, 0)], ['Tricastin-2', 915, 2030, 99, datetime.datetime(2019, 1, 1, 0, 0), datetime.datetime(2019, 4, 10, 0, 0)], ['Tricastin-3', 915, 2035, 99, datetime.datetime(2019, 1, 1, 0, 0), datetime.datetime(2019, 4, 10, 0, 0)], ['Tricastin-4', 915, 2035, 99, datetime.datetime(2019, 1, 1, 0, 0), datetime.datetime(2019, 4, 10, 0, 0)], ['Dampierre-1', 890, 2039, 99, datetime.datetime(2019, 1, 1, 0, 0), datetime.datetime(2019, 4, 10, 0, 0)], ['Dampierre-2', 890, 2040, 99, datetime.datetime(2019, 1, 1, 0, 0), datetime.datetime(2019, 4, 10, 0, 0)], ['Dampierre-3', 890, 2040, 99, datetime.datetime(2019, 2, 1, 0, 0), datetime.datetime(2019, 5, 11, 0, 0)], ['Dampierre-4', 890, 2040, 99, datetime.date

[['Bugey-4', 880, 2039, 99, datetime.datetime(2019, 1, 1, 0, 0), datetime.datetime(2019, 4, 10, 0, 0)], ['Bugey-5', 880, 2040, 99, datetime.datetime(2019, 1, 1, 0, 0), datetime.datetime(2019, 4, 10, 0, 0)], ['Tricastin-1', 915, 2030, 99, datetime.datetime(2019, 1, 1, 0, 0), datetime.datetime(2019, 4, 10, 0, 0)], ['Tricastin-2', 915, 2030, 99, datetime.datetime(2019, 1, 1, 0, 0), datetime.datetime(2019, 4, 10, 0, 0)], ['Tricastin-3', 915, 2035, 99, datetime.datetime(2019, 1, 1, 0, 0), datetime.datetime(2019, 4, 10, 0, 0)], ['Tricastin-4', 915, 2035, 99, datetime.datetime(2019, 1, 1, 0, 0), datetime.datetime(2019, 4, 10, 0, 0)], ['Dampierre-1', 890, 2039, 99, datetime.datetime(2019, 1, 1, 0, 0), datetime.datetime(2019, 4, 10, 0, 0)], ['Dampierre-2', 890, 2040, 99, datetime.datetime(2019, 1, 1, 0, 0), datetime.datetime(2019, 4, 10, 0, 0)], ['Dampierre-3', 890, 2040, 99, datetime.datetime(2019, 2, 1, 0, 0), datetime.datetime(2019, 5, 11, 0, 0)], ['Dampierre-4', 890, 2040, 99, datetime.date

[['Bugey-5', 880, 2040, 99, datetime.datetime(2019, 1, 1, 0, 0), datetime.datetime(2019, 4, 10, 0, 0)], ['Dampierre-2', 890, 2040, 99, datetime.datetime(2019, 1, 1, 0, 0), datetime.datetime(2019, 4, 10, 0, 0)], ['Dampierre-3', 890, 2040, 99, datetime.datetime(2019, 1, 1, 0, 0), datetime.datetime(2019, 4, 10, 0, 0)], ['Dampierre-4', 890, 2040, 99, datetime.datetime(2019, 1, 1, 0, 0), datetime.datetime(2019, 4, 10, 0, 0)], ['Gravelines-3', 910, 2041, 99, datetime.datetime(2019, 1, 1, 0, 0), datetime.datetime(2019, 4, 10, 0, 0)], ['Gravelines-4', 910, 2041, 99, datetime.datetime(2019, 1, 1, 0, 0), datetime.datetime(2019, 4, 10, 0, 0)], ['Gravelines-5', 910, 2045, 99, datetime.datetime(2019, 2, 1, 0, 0), datetime.datetime(2019, 5, 11, 0, 0)], ['Gravelines-6', 910, 2045, 99, datetime.datetime(2019, 2, 1, 0, 0), datetime.datetime(2019, 5, 11, 0, 0)], ['St-Laurent-B1', 915, 2042, 99, datetime.datetime(2019, 2, 1, 0, 0), datetime.datetime(2019, 5, 11, 0, 0)], ['St-Laurent-B2', 915, 2042, 99, d

[['Flamanville-3', 1650, 2083, 62, datetime.datetime(2019, 1, 1, 0, 0), datetime.datetime(2019, 3, 4, 0, 0)], ['Cattenom-3', 1300, 2051, 99, datetime.datetime(2019, 2, 1, 0, 0), datetime.datetime(2019, 5, 11, 0, 0)], ['Cattenom-4', 1300, 2052, 99, datetime.datetime(2019, 2, 1, 0, 0), datetime.datetime(2019, 5, 11, 0, 0)], ['Penly-1', 1330, 2050, 99, datetime.datetime(2019, 2, 4, 5, 0), datetime.datetime(2019, 5, 14, 5, 0)], ['Penly-2', 1330, 2052, 99, datetime.datetime(2019, 3, 1, 0, 0), datetime.datetime(2019, 6, 8, 0, 0)], ['Golfech-1', 1310, 2051, 99, datetime.datetime(2019, 3, 1, 0, 0), datetime.datetime(2019, 6, 8, 0, 0)], ['Golfech-2', 1310, 2053, 99, datetime.datetime(2019, 3, 1, 0, 0), datetime.datetime(2019, 6, 8, 0, 0)], ['Chooz-B1', 1500, 2056, 99, datetime.datetime(2019, 4, 1, 0, 0), datetime.datetime(2019, 7, 9, 0, 0)], ['Chooz-B2', 1500, 2057, 99, datetime.datetime(2019, 4, 14, 3, 0), datetime.datetime(2019, 7, 22, 3, 0)], ['Civaux-1', 1495, 2058, 99, datetime.datetime(20

[['Flamanville-3', 1650, 2083, 62, datetime.datetime(2019, 2, 1, 0, 0), datetime.datetime(2019, 4, 4, 0, 0)], ['EPR2', 1670, 2100, 62, datetime.datetime(2019, 2, 1, 0, 0), datetime.datetime(2019, 4, 4, 0, 0)], ['EPR2', 1670, 2100, 62, datetime.datetime(2019, 2, 7, 6, 0), datetime.datetime(2019, 4, 10, 6, 0)], ['EPR2', 1670, 2100, 62, datetime.datetime(2019, 3, 1, 0, 0), datetime.datetime(2019, 5, 2, 0, 0)], ['EPR2', 1670, 2100, 62, datetime.datetime(2019, 3, 1, 0, 0), datetime.datetime(2019, 5, 2, 0, 0)], ['EPR2', 1670, 2100, 62, datetime.datetime(2019, 4, 1, 0, 0), datetime.datetime(2019, 6, 2, 0, 0)], ['EPR2', 1670, 2100, 62, datetime.datetime(2019, 4, 1, 0, 0), datetime.datetime(2019, 6, 2, 0, 0)], ['EPR2', 1670, 2100, 62, datetime.datetime(2019, 4, 1, 0, 0), datetime.datetime(2019, 6, 2, 0, 0)], ['EPR2', 1670, 2100, 62, datetime.datetime(2019, 4, 6, 16, 0), datetime.datetime(2019, 6, 7, 16, 0)], ['EPR2', 1670, 2100, 62, datetime.datetime(2019, 5, 1, 0, 0), datetime.datetime(2019, 7

# 2. Neighbours in Europe

In [6]:
L_parc_hist_E=[]
Parc_hist_E_df=pd.read_csv("Nucleaire_historique_Europe.csv",sep=';',decimal=',')
for i in Parc_hist_E_df.index:
    row=list(Parc_hist_E_df.loc[i])
    L_parc_hist_E.append(row)

In [7]:
L_new_nuc_E=[]
New_nuc_E_df=pd.read_csv("Nouveau_nucleaire_Europe.csv",sep=';',decimal=',')
for i in New_nuc_E_df.index:
    row=list(New_nuc_E_df.loc[i])
    L_new_nuc_E.append(row)

In [8]:
# Key functions
L_modul_year=[1.2,1.1,1,0.95,0.95,0.9,0.95,0.93,0.9,1,1.02,1.1]
taux_unav_hist=0.27
taux_unav_new=0.17
Power_EPR2=1670
Power_SMR={'GB':470}

def duration_unav_E(row):
    if row[1] in ['HINKLEY POINT-C1','HINKLEY POINT-C2']:
        return round(365*taux_unav_new)
    else:
        return round(365*taux_unav_hist)

def parc_area(L_to_filter,area):
    L=[]
    for x in L_to_filter:
        if x[0]==area:
            L.append(x)
    return L

def complete_parc_E(L_parc_hist,L_new_nuc,year):
    L_parc_year=[]
    for row in L_parc_hist:
        if year<=row[3]:
            L_parc_year.append(row+[duration_unav(row),None,None])
    
    S_areas_new_nuc=set([x[0] for x in L_new_nuc])
    for area in S_areas_new_nuc:
        L=parc_area(L_new_nuc,area)
        
        if year<L[0][1]:
            N_EPR2=0
            N_SMR=0
        elif year>=L[-1][1]:
            N_EPR2=L[-1][2]
            N_SMR=L[-1][3]
        else:
            N_EPR2=L[year-L[0][1]][2]
            N_SMR=L[year-L[0][1]][3]
    
    for i in range(N_EPR2):
        L_parc_year.append([area,'EPR2',Power_EPR2,2100]+[round(365*taux_unav_new),None,None])
    for i in range(N_SMR):
        L_parc_year.append([area,'SMR',Power_SMR[area],2100]+[round(365*taux_unav_new),None,None])
    
    return L_parc_year


L_month=[31,28,31,30,31,30,31,31,30,31,30,31]
def goal_month_E(L_parc_area,L_modul_year):
    P_moy=sum([x[2]*(365-x[4])/365 for x in L_parc_area])
    return [L_month[m]*P_moy*24*L_modul_year[m] for m in range(12)]

def prod_month_E(L_parc_area,m):# m in [0..11] here
    E=0
    d0=dt.datetime(2019,m+1,1)
    if m<11:
        d1=dt.datetime(2019,m+2,1)
    else:
        d1=dt.datetime(2020,1,1)
    for x in L_parc_area:
        if x[5]==None or x[5]>=d1 or x[6]<=d0:
            E+=x[2]*L_month[m]*24
        else:
            delta=min(d1,x[6])-max(d0,x[5])
            E+=(L_month[m]*24-(delta.days*24+delta.seconds/3600))*x[2]
    return E

d2020=dt.datetime(2020,1,1)
d1h=dt.timedelta(hours=1)
def optimize_E(L_parc_hist,L_new_nuc,L_modul_year,year,delta_plus=0.01,delta_minus=0.01):
    L_parc_year=complete_parc_E(L_parc_hist,L_new_nuc,year)
    S_areas=set([x[0] for x in L_parc_year])
    
    L_ret=[]
    for area in S_areas:
        L_parc_area=parc_area(L_parc_year,area)
        L_goals=goal_month_E(L_parc_area,L_modul_year)
        for m in range(12):
            d0=dt.datetime(2019,m+1,1)
            if m<11:
                d1=dt.datetime(2019,m+2,1)
            else:
                d1=d2020
            E_prod=prod_month_E(L_parc_area,m)
            i=0
            while i<len(L_parc_area) and E_prod>(1+delta_plus)*L_goals[m]:
                if L_parc_area[i][5]!=None:
                    i+=1
                else:
                    delta_arret_tranche=dt.timedelta(days=L_parc_area[i][4])
                    L_parc_area[i][5]=d0
                    L_parc_area[i][6]=d0+delta_arret_tranche
                    if L_parc_area[i][6]>d2020:
                        delta_depasse=L_parc_area[i][6]-d2020
                        L_parc_area[i][5]-=delta_depasse
                        L_parc_area[i][6]=d2020
                    E_prod=prod_month_E(L_parc_area,m)
                    while E_prod<(1-delta_minus)*L_goals[m] and L_parc_area[i][5]<d2020:
                        L_parc_area[i][5]+=d1h
                        L_parc_area[i][6]+=d1h
                        E_prod=prod_month_E(L_parc_area,m)
                    i+=1
        L_ret+=L_parc_area
    return L_ret

def nuc_capacity_E(L_parc_area):
    P_old_nuc=0
    P_new_nuc=0
    for x in L_parc_area:
        if x[1] in ['EPR2','SMR','HINKLEY POINT-C1','HINKLEY POINT-C2']:
            P_new_nuc+=x[2]
        else:
            P_old_nuc+=x[2]
    return P_new_nuc, P_old_nuc

d2019=dt.datetime(2019,1,1)
def nuc_availability_E(L_parc_area):
    P_new_nuc, P_old_nuc=nuc_capacity_E(L_parc_area)
    
    L_dates=[]
    L_new_nuc=[0 for i in range(8760)]
    L_old_nuc=[0 for i in range(8760)]
    date=d2019
    for i in range(8760):
        L_dates.append(pd.Timestamp(date))
        date+=d1h
        for x in L_parc_area:
            if not x[5]<=L_dates[-1]<=x[6]:
                if x[1] in ['EPR2','SMR','HINKLEY POINT-C1','HINKLEY POINT-C2']:
                    L_new_nuc[i]+=x[2]
                else:
                    L_old_nuc[i]+=x[2]
                    
    L_new_nuc=[x/max(P_new_nuc,1) for x in L_new_nuc]
    L_old_nuc=[x/max(P_old_nuc,1) for x in L_old_nuc]
    
    return pd.DataFrame({"Date":L_dates,"New nuke":L_new_nuc,"Old nuke":L_old_nuc})


In [9]:
L_parc_year=optimize_E(L_parc_hist_E,L_new_nuc_E,L_modul_year,2040)
L_parc_year

[['GB',
  'HINKLEY POINT-C1',
  1630,
  2087,
  99,
  datetime.datetime(2019, 1, 18, 15, 0),
  datetime.datetime(2019, 4, 27, 15, 0)],
 ['GB',
  'HINKLEY POINT-C2',
  1630,
  2087,
  99,
  datetime.datetime(2019, 3, 12, 22, 0),
  datetime.datetime(2019, 6, 19, 22, 0)],
 ['GB',
  'SIZEWELL-B',
  1188,
  2055,
  99,
  datetime.datetime(2019, 5, 1, 0, 0),
  datetime.datetime(2019, 8, 8, 0, 0)],
 ['GB',
  'EPR2',
  1670,
  2100,
  62,
  datetime.datetime(2019, 5, 26, 12, 0),
  datetime.datetime(2019, 7, 27, 12, 0)],
 ['GB',
  'EPR2',
  1670,
  2100,
  62,
  datetime.datetime(2019, 7, 22, 18, 0),
  datetime.datetime(2019, 9, 22, 18, 0)],
 ['GB',
  'EPR2',
  1670,
  2100,
  62,
  datetime.datetime(2019, 8, 6, 14, 0),
  datetime.datetime(2019, 10, 7, 14, 0)],
 ['GB',
  'SMR',
  470,
  2100,
  62,
  datetime.datetime(2019, 9, 1, 0, 0),
  datetime.datetime(2019, 11, 2, 0, 0)],
 ['GB',
  'SMR',
  470,
  2100,
  62,
  datetime.datetime(2019, 9, 15, 5, 0),
  datetime.datetime(2019, 11, 16, 5, 0)],

In [11]:
dict_capa_old={'Year':[2030,2040,2050,2060],'GB':[],'ES':[],'CH':[],'BE':[]}
dict_capa_new={'Year':[2030,2040,2050,2060],'GB':[],'ES':[],'CH':[],'BE':[]}
for year in [2030,2040,2050,2060]:
    L_parc_year=optimize_E(L_parc_hist_E,L_new_nuc_E,L_modul_year,year)
    print(L_parc_year)
    for area in ['GB','ES','CH','BE']:
        L_parc_area=parc_area(L_parc_year,area)
        P_new, P_old=nuc_capacity_E(L_parc_area)
        dict_capa_new[area].append(P_new)
        dict_capa_old[area].append(P_old)
        Availability_df=nuc_availability_E(L_parc_area)
        Availability_df.to_csv("Prod_Europe/availability_nuclear_"+area+"_"+str(year)+".csv",sep=";",decimal=".",index=False)

df_capa_old=pd.DataFrame(dict_capa_old).set_index('Year')
df_capa_new=pd.DataFrame(dict_capa_new).set_index('Year')
df_capa_old.to_csv("Prod_Europe/Nuclear_capacity_old.csv",sep=';',decimal=',')
df_capa_new.to_csv("Prod_Europe/Nuclear_capacity_new.csv",sep=';',decimal='.')

[['BE', 'DOEL-4', 1047, 2035, 99, datetime.datetime(2019, 1, 23, 16, 0), datetime.datetime(2019, 5, 2, 16, 0)], ['BE', 'TIHANGE-3', 1054, 2035, 99, datetime.datetime(2019, 5, 14, 6, 0), datetime.datetime(2019, 8, 21, 6, 0)], ['GB', 'HEYSHAM-B1', 620, 2030, 99, datetime.datetime(2019, 1, 1, 0, 0), datetime.datetime(2019, 4, 10, 0, 0)], ['GB', 'HEYSHAM-B2', 620, 2030, 99, datetime.datetime(2019, 1, 24, 20, 0), datetime.datetime(2019, 5, 3, 20, 0)], ['GB', 'HINKLEY POINT-C1', 1630, 2087, 99, datetime.datetime(2019, 3, 25, 11, 0), datetime.datetime(2019, 7, 2, 11, 0)], ['GB', 'HINKLEY POINT-C2', 1630, 2087, 99, datetime.datetime(2019, 5, 30, 2, 0), datetime.datetime(2019, 9, 6, 2, 0)], ['GB', 'SIZEWELL-B', 1188, 2055, 99, datetime.datetime(2019, 7, 29, 21, 0), datetime.datetime(2019, 11, 5, 21, 0)], ['ES', 'ASCO-1', 995, 2030, 99, datetime.datetime(2019, 1, 10, 16, 0), datetime.datetime(2019, 4, 19, 16, 0)], ['ES', 'ASCO-2', 997, 2031, 99, datetime.datetime(2019, 3, 18, 18, 0), datetime.da